In [5]:
import requests
import pandas as pd
import numpy as np
import os
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import time
import pyarrow as pa
from urllib.parse import quote_plus
from pandas.api.types import is_string_dtype
import gc
import warnings

import config
import config_media_costs
from normalize_funcs import normalize_columns_types, append_custom_columns
from db_funcs import createDBTable, downloadTableToDB, get_mssql_table, removeRowsFromDB
from create_dicts import get_cleaning_dict, get_media_discounts
from create_dicts_adex import get_adex_dicts

# start_of_the_time = config_tv_invest.start_of_the_time # указываем дату начала сбора данных, для преобразования номера месяца

start_date = '2025-02-01'#'2023-01-01'
start_date = datetime.strptime(start_date, '%Y-%m-%d').date()

end_date = '2025-04-02'
end_date = datetime.strptime(end_date, '%Y-%m-%d').date()

print(f'start_date: {start_date} / end_date: {end_date}')

# filters_lst = [config.article_lev_4_id_str, config.subbrand_id_str]


start_date: 2025-02-01 / end_date: 2025-04-02


In [6]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

warnings.simplefilter(action='ignore', category=FutureWarning)
# убираем лишние предупреждения
pd.set_option('mode.chained_assignment', None)

sep_str = '*' * 50

In [26]:
# создаем функцию, которая вернет номер месяца от начальной даты отсчета
def get_mon_num_from_date(curr_date):
    curr_date = datetime.strptime(str(curr_date), '%Y-%m-%d')
    # с помощью метода relativedelta получаем кол-во лет и месяцев от даты начала до текущей даты
    relative_date = relativedelta(curr_date, config_media_costs.start_of_the_time)
    # теперь нужно получить номер текущего месяца от даты начала
    months_count = relative_date.years * 12 + relative_date.months
    
    return months_count

In [18]:
def get_media_costs_report(start_date='', end_date='', media_type='tv', flag='regular'):
    start_time = datetime.now()
    print(f'Скрипт запущен {start_time}')
    
    if flag.lower()=='first':
        table_name = config_media_costs.media_dicts_costs[media_type][0]
        vars_lst = config_media_costs.media_dicts_costs[media_type][1]
        
        createDBTable(config.db_name, table_name, vars_lst, flag='create')
        # создаем пустые словари справочников через цикл
        # забираем из файла create_dicts - словарь, где 
        # ключ - это название таблицы в Медиаскоп (для дальнейшего удобства так сделано)
        # значение - это список, который содержит:
        # [0] - название таблицы в БД
        # [1] - список полей с типами данных для БД
        # [2] - список полей с целочисленными значениями для нормализации
        # [3] - поле, по которому выполняется фильтрация в исходной БД Медиаскоп при ОБНОВЛЕНИИ справочника
        for value in config_media_costs.adex_ad_lst_dicts.values():
            createDBTable(config.db_name, value[0] , value[1], flag='create')



    cur_date = datetime.now().date()
    cur_year_month = cur_date.strftime('%Y-%m')
    
# если дата начала задана, то приводим ее к формату Даты
    start_date = datetime.strptime(str(start_date), '%Y-%m-%d').date() if start_date else ''
# забираем год-месяц для проверки относительно текущего месяца
    start_year_month = start_date.strftime('%Y-%m') if start_date else ''
    
# если дата окончания задана, то приводим ее к формату Даты
    end_date = datetime.strptime(str(end_date), '%Y-%m-%d').date() if end_date else ''
# забираем год-месяц для проверки относительно текущего месяца
    end_year_month = end_date.strftime('%Y-%m') if end_date else ''
    
# если дата начала НЕ задана, значит - это ежемесячное обновление
# от текущей даты берем 2 месяца назад
# удаляем из БД эти месяцы
# далее загрузим новые данные - до Текущий месяц минус один (в Медиаское запоздание на 1 месяц)
    
    if not start_date or start_year_month==cur_year_month:
        start_date = (cur_date  - relativedelta(months=2))
        
# если даты окончания нет или она равна текущему месяцу, то задаем прошлый месяц
    if not end_date or end_year_month==cur_year_month:
        end_date = (cur_date  - relativedelta(months=1))
        
# получаем номер месяца начала загрузки данных по летоисчислению Медиаскоп
    start_mon = get_mon_num_from_date(start_date)
    
    # если это НЕ первая загрузка, то удаляем строки из БД начиная с даты начала текущей загрузки
    # для отчетов Buying - расходы приходят с запозданием на неделю, а так же страхуемся от возможных дублей в БД при новой загрузке
    if flag=='regular':
        cond = f'mon_num >= {start_mon}'
            
        print()
        print(sep_str)
        print(f'Удалем строки из таблицы: media_{media_type}_costs по условию: {cond}')
        print()
    
        removeRowsFromDB(config.db_name, f'media_{media_type}_costs', cond)
        print()

    # считаем кол-во месяцев в периоде
    # каждый месяц мы будем забирать по отдельности и записывать его в БД
    count_months = relativedelta(end_date, start_date).months + 1
    
    print()
    print(f'Загружаем отчет за период {start_date} - {end_date}. Общее количество месяцев: {count_months}')
    print(sep_str)
    print()
    
    # проходимся по общему количеству дней
    for i in range(count_months):
        # формируем отдельную дату для загрузки
        cur_date = start_date + relativedelta(months=i)
        cur_mon = get_mon_num_from_date(cur_date)
        print()
        print(f'{"="*10}Загружаем {cur_date}. Статистика по {media_type}')
        print()
        
        table_name = config_media_costs.media_dicts_costs[media_type][0]
        int_lst = config_media_costs.media_dicts_costs[media_type][2]
        float_lst = config_media_costs.media_dicts_costs[media_type][3]
        
        df = get_table_costs(int_lst, 
                           float_lst, 
                           start_date=str(cur_date), 
                           mon_num=int(cur_mon), 
                           media_type=media_type)

        downloadTableToDB(config.db_name, table_name, df)
                      

In [ ]:
# start_date='2023-01-01'
# end_date='2023-02-01'
# for media_type in config_media_costs.media_type_lst:
#     get_media_costs_report(start_date=start_date, end_date=end_date, media_type=media_type, flag='first')

0


In [ ]:
int_lst = config_media_costs.media_tv_costs_int_lst
float_lst = config_media_costs.media_tv_costs_float_lst
df = get_table_costs(int_lst, float_lst, start_date='2025-03-01', mon_num=422, media_type='tv')

In [ ]:
df.shape

In [ ]:
df['adId'].drop_duplicates().shape

In [ ]:
df_ad_dict = df[['adId']].drop_duplicates()

In [ ]:
# забираем из БД из справочника объявлений уникальные ИД
query = f"select distinct adId  from media_tv_costs"
ad_id_dict = get_mssql_table(config.db_name, query=query)
# создаем список Уникальных ИД Объявлений, которые уже есть в справочнике в БД
ad_id_lst = list(set(list(df_ad_dict['adId'])) - set(list(ad_id_dict['adId'])))
# оставляем только те объявления, которых нет в справочнике
df_ad_dict = df_ad_dict.query('adId in @ad_id_lst')

In [ ]:
df_ad_dict.shape

In [ ]:
ad_id_dict.shape

In [ ]:
len(ad_id_lst)

In [19]:
get_media_costs_report(start_date='2025-02-01', end_date='2025-04-01', media_type='tv', flag='first')

Скрипт запущен 2025-04-10 09:30:21.608509
Все ок. Подключились!
Пустая таблица media_tv_costs успешно создана в БД mediascope_x5
Все ок. Подключились!
Пустая таблица adex_ad_dict_list_tv успешно создана в БД mediascope_x5

Загружаем отчет за период 2025-02-01 - 2025-03-10. Общее количество месяцев: 2
**************************************************


==========Загружаем 2025-02-01. Статистика по tv

Все ок. Подключились!
Загрузка завершена успешно
Все ок. Подключились!
Скрипт запущен 2025-04-10 09:30:35.243808
Данные добавлены в БД: mediascope_x5, таблица: media_tv_costs
Скрипт отработал 2025-04-10 09:30:43.187749
Время выполнения задачи: 0:00:07.943941
Загрузка завершена. Данные успешно добавлены в БД: mediascope_x5
##################################################

Все ок. Подключились!
Загрузка завершена успешно
Все ок. Подключились!
Загрузка завершена успешно

==========Загружаем 2025-03-01. Статистика по tv

Все ок. Подключились!
Загрузка завершена успешно
Все ок. Подключились!

In [ ]:
df_ad_dict.head(1)

In [ ]:
ad_id_str = config.get_lst_to_str(df_ad_dict)

In [12]:
# ТВ расходы - создаем функцию, которая 
# - забирает расходы по месяцам
# - переименовывает поля (приводит к стандарту ТВ Индекс
# Функция принимает на вход
# *normalize_lst - передаем, как отдельные параметры - Список полей с типом int / Список полей с типом float
# start_mon - дата начала месяца, за который забираем статистику
# mon_num - номер месяца по летоисчислению Медиаскоп
# media_type - тип медиа, по которому забираем расходы

def get_table_costs(*normalize_lst, start_date='2023-01-01', mon_num=396, media_type='tv'):
    # для запроса к БД приводим тип медиа к нижнему регистру
    media_type = media_type.lower()
    # Формируем запрос к БД
    # В зависимости от типа медиа меняется первая строка в запросе - поля, которые нам нужны для таблицы Фактов
    # названия полей находятся в справочнике config_tv_investments.first_row_query_dict
    # Фильтрация строк для ВСЕХ одинаковая производится по условиям config_tv_investments.main_filter_str
    query = f"""select 
    {config_media_costs.first_row_query_dict[media_type]}
    from {media_type}_Ad_month t1 left join {media_type}_Appendix t10 
    on t1.vid=t10.vid
    left join {media_type}_Ad t3
    on t1.vid=t3.vid
    where t1.mon={str(mon_num)}  and 
    ({config_media_costs.main_filter_str})"""
    # отправляем запрос в БД Медиа инвестиции

    df = get_mssql_table(config.investments_db_name, query=query, conn_lst=config.conn_lst)
    
    # переименовываем поля - приводим их в соответсвии с названиями из ТВ Индекс
    # перебираем справочник config_tv_investments.rename_cols_dict
    # если название поля из Медиа инвестиции есть в ключах, то забираем пару ключ-значение
    # чтобы передать для присвоения нового названия
    new_cols_name = {key: value for (key, value) in config_media_costs.rename_cols_dict.items() if key in list(df.columns)}
    df = df.rename(columns=new_cols_name)
    df['researchDate'] = start_date
    df['ConsolidatedCostRUB'] = df['ConsolidatedCostRUB'].astype('int64')
    # вызываем функцию, чтобы добавить новые поля, дисконт и расходы с дисконтом
    # Добавляем в датаФрейм - год, тип медиа, тип медиа+ИД объявления, расходы с дисконтом
    df = append_custom_columns(df, report='investments', media_type=media_type)
    # нормализуем типы данных
    df = normalize_columns_types(df, normalize_lst[0], normalize_lst[1])
    return df

In [ ]:
df.shape

In [ ]:
df.head(1)

In [ ]:
createDBTable(config.db_name, config_tv_investments.investments_tv_costs , config_tv_investments.investments_tv_costs_vars_list, flag='create')

In [ ]:
downloadTableToDB(config.db_name, config_tv_investments.investments_tv_costs, df)

In [ ]:
config.investments_db_name

In [ ]:
query = f"""select 
    t1.vid, t2.bid, t3.blid
    from tv_Ad_month t1 left join tv_Appendix t2 
    on t1.vid=t2.vid
    left join tv_Ad t3
    on t1.vid=t3.vid
    where t1.mon='396' and t1.vid='4404947'  and 
    (t2.sid2 in (2408, 4780, 4926, 5028) or t2.sbid in (137166, 489562, 494633, 556193, 872285, 966983, 1155675, 1245528))
    """
    df = get_mssql_table(config.investments_db_name, query=query, conn_lst=config.conn_lst)

In [ ]:
query = f"""select 
t1.vid, t2.bid, t3.blid
from tv_Ad_month t1 left join tv_Appendix t2 
on t1.vid=t2.vid
left join tv_Ad t3
on t1.vid=t3.vid
where t1.mon='396' and t1.vid='4404947'  and (t2.sid2 in ({config.article_lev_4_id_str}) or t2.sbid in ({config.subbrand_id_str}))
"""
df = get_mssql_table(config.investments_db_name, query=query, conn_lst=config.conn_lst)

In [ ]:
df.shape

In [ ]:
start_date = '2023-04-08'

In [ ]:
start_date = '2025-04-01'
start_date = datetime.strptime(str(start_date), '%Y-%m-%d').date()
start_year_month = start_date.strftime('%Y-%m')

curr_date = datetime.now().date()
cur_year_month = curr_date.strftime('%Y-%m')

if start_year_month==cur_year_month:
    start_date = curr_date - relativedelta(months=4)
    start_mon = get_mon_num_from_date(start_date)

    print(start_date)
    print(start_mon)


In [ ]:
get_media_costs_report(start_date='2023-01-01', end_date='2023-02-01', media_type='tv', flag='first')

In [ ]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

warnings.simplefilter(action='ignore', category=FutureWarning)
# убираем лишние предупреждения
pd.set_option('mode.chained_assignment', None)

sep_str = '*' * 50

In [ ]:
print(get_mon_num_from_date(start_date, start_of_the_time))

In [ ]:
query = f"select * from tv_Ad where vid='4404947'"

In [ ]:
df = get_mssql_table(config.investments_db_name, query=query, conn_lst=config.conn_lst)

In [ ]:
df

In [ ]:
query = f"select * from BrandList where blid='2046493'"
df = get_mssql_table(config.investments_db_name, query=query, conn_lst=config.conn_lst)

In [ ]:
df

In [ ]:
query = f"select * from tv_Appendix where vid='4404947'"
df = get_mssql_table(config.investments_db_name, query=query, conn_lst=config.conn_lst)

In [ ]:
df

In [ ]:
query = f"select * from Brand where bid='568656'"
df = get_mssql_table(config.investments_db_name, query=query, conn_lst=config.conn_lst)

In [ ]:
df

In [ ]:
query = f"""select 
* 
from tv_Ad_month t1 
where t1.mon='396' and t1.vid='4404947'  
"""
df = get_mssql_table(config.investments_db_name, query=query, conn_lst=config.conn_lst)

In [ ]:
df.shape

In [ ]:
left join tv_Appendix t2 
on t1.vid=t2.vid
(t2.sid2 in (2408, 4780, 4926, 5028) or t2.sbid in (137166, 489562, 494633, 556193, 872285, 966983, 1155675, 1245528))
and t2.bid='568656'

In [ ]:
query = f"""select 
t1.vid, t2.bid, t3.blid
from tv_Ad_month t1 left join tv_Appendix t2 
on t1.vid=t2.vid
left join tv_Ad t3
on t1.vid=t3.vid
where t1.mon='396' and t1.vid='4404947'  and 
(t2.sid2 in (2408, 4780, 4926, 5028) or t2.sbid in (137166, 489562, 494633, 556193, 872285, 966983, 1155675, 1245528))
"""
df = get_mssql_table(config.investments_db_name, query=query, conn_lst=config.conn_lst)

In [ ]:
df.shape

In [ ]:
df.head(2)

In [ ]:
df['bid'].drop_duplicates()

In [ ]:
for value in config_media_costs.adex_ad_lst_dicts.values():
    print(value[0])
    print(value[1])

In [ ]:
value[1]

In [ ]:

segments_main_link = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vR2WJjY1kEsKluiXddRiFrqKFOVGO06eJY4CVt33OhJ7KdALHHr3sYsqXyeoVxmNI08VNUP6mAlnQKt/pub?gid=0&single=true&output=csv'

media_type_detail_link = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTmkq1SXPqmHqCx9MfiMlbf3nV3PRKctT0r1RItnLxnKt7MYzBg56V99obmqbjWek3ux8gdjhvLQR8E/pub?gid=0&single=true&output=csv'

tv_net_group_link = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTvelFTRLTMGnlLWLmXvLXRLVdBe7magXnTfncl72nXYN0l6s2NfPLLe-M2ipdpV94JesJ-ZuA6lejj/pub?gid=0&single=true&output=csv'

In [ ]:
df = get_mssql_table(db_name, table_name='', query='')